In [1]:
pip install rdflib
pip install transliterate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.3 MB/s eta 0:00:00


In [193]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph, RDF, XSD
from transliterate import translit
import pandas as pd

In [194]:
g = Graph()
g.parse('kish.rdf')

<Graph identifier=N341e5971a3e34a47b9c119425b67033a (<class 'rdflib.graph.Graph'>)>

In [196]:
# Посмотрим, есть ли какие-либо триплеты в нашем графе

all_classes = g.query("""
    ASK WHERE {?s ?p ?o}
""")
for i in all_classes:
  print(i)

True


In [197]:
# Посмотрим на все доступные триплеты
all = g.query("""
  SELECT ?subject ?predicate ?object
  WHERE {?subject ?predicate ?object}

""")
for row in all:
  print(row)

(rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11#Track'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))
(rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/kish#Album_name'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#DatatypeProperty'))
(rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11#Track'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11#Album'))
(rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/kish#contains'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty'))
(rdflib.term.URIRef('http://www.semanticweb.o

In [198]:
# Classes
album_class = URIRef("http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11#Album")
track_class = URIRef("http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11#Track")

# Properties
cont = URIRef("http://www.semanticweb.org/mosto/ontologies/kish#contains")

# Attributes
album_name_attr = URIRef("http://www.semanticweb.org/mosto/ontologies/kish#Album_name")
track_name_attr = URIRef("http://www.semanticweb.org/mosto/ontologies/kish#Track_name")

In [199]:
def new_album(album_name):
  new_album = URIRef(f"http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/{album_name}")
  g.add((new_album, RDF.type, album_class))
  g.add((new_album, album_name_attr, Literal(album_name, lang="en")))

def new_track(track_name, album_name):
  new_track = URIRef(f"http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/{track_name}")
  new_album = URIRef(f"http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/{album_name}")
  g.add((new_track, RDF.type, track_class))
  g.add((new_track, track_name_attr, Literal(track_name, lang="en")))
  g.add((new_album, cont, new_track))


In [200]:
all_info = pd.read_csv('korol_i_shut_songs.csv', sep=';')

In [202]:
all_info

,group_name,album_name,year,track
0,Король и Шут,Акустический альбом,1999,Кукла колдуна
1,Король и Шут,Акустический альбом,1999,Наблюдатель
2,Король и Шут,Акустический альбом,1999,Бедняжка
3,Король и Шут,Акустический альбом,1999,Прыгну со скалы
4,Король и Шут,Акустический альбом,1999,Девушка и Граф
...,...,...,...,...
164,Король и Шут,Жаль - нет ружья!,2002,Вино Хоббитов
165,Король и Шут,Жаль - нет ружья!,2002,Разборки из-за Баб
166,Король и Шут,Жаль - нет ружья!,2002,Утопленник
167,Король и Шут,Жаль - нет ружья!,2002,Медведь


In [201]:
all_info.album_name.unique()

array(['Акустический альбом', 'Бунт на корабле', 'Как в старой сказке',
       'Камнем по Голове', 'Король и Шут', 'Продавец Кошмаров',
       'Театр Демона', 'Тень Клоуна', 'TODD - Акт 1. «Праздник Крови»',
       'TODD - Акт 2. «На Краю»', 'Жаль - нет ружья!'], dtype=object)

In [89]:
all_info['album_name'] + ' ' + '(' + all_info['year'].astype(str) + ')'

0      Акустический альбом (1999)
1      Акустический альбом (1999)
2      Акустический альбом (1999)
3      Акустический альбом (1999)
4      Акустический альбом (1999)
                  ...            
164      Жаль - нет ружья! (2002)
165      Жаль - нет ружья! (2002)
166      Жаль - нет ружья! (2002)
167      Жаль - нет ружья! (2002)
168      Жаль - нет ружья! (2002)
Length: 169, dtype: object

In [203]:
grouped_albums = all_info.groupby('album_name', as_index=False).agg({'track': list})

In [204]:
grouped_albums.album_name = grouped_albums.album_name.apply(lambda x: '_'.join(x.split(' ')))
grouped_albums.track = grouped_albums.track.apply(lambda x: ['_'.join(i.split(' ')) for i in x])

In [205]:
for i in grouped_albums.album_name.tolist():
  new_album(i)
  for k in grouped_albums.loc[grouped_albums['album_name'] == i]['track'].item():
    new_track(k, i)

In [206]:
g.serialize(destination="kish_albums_tracks.rdf")

<Graph identifier=N341e5971a3e34a47b9c119425b67033a (<class 'rdflib.graph.Graph'>)>

### Данные добавлены, с помощью запросов посмотрим, что получилось


In [207]:
# Посмотрим на все доступные триплеты
all = g.query("""
  SELECT ?subject ?predicate ?object
  WHERE {?subject ?predicate ?object}
  LIMIT 10
""")
for row in all:
  print(row)

(rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Смерть_Ловетт_(Ария_Ловетт)'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11#Track'))
(rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Паника_в_селе'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11#Track'))
(rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Охотник'), rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/kish#Track_name'), rdflib.term.Literal('Охотник', lang='en'))
(rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/Акустический_альбом'), rdflib.term.URIRef('http://www.w3.org/1999

In [208]:
# Посмотрим на все доступные предикаты
all_preds = g.query("""
  SELECT DISTINCT ?predicate
  WHERE {?subject ?predicate ?object}
""")
for row in all_preds:
  print(row)

(rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),)
(rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/kish#Track_name'),)
(rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/kish#contains'),)
(rdflib.term.URIRef('http://www.semanticweb.org/mosto/ontologies/kish#Album_name'),)
(rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),)


In [209]:
# Посмотрим на все имеющиеся альбомы
all_albums = g.query("""
    SELECT ?album
    WHERE {
        ?album rdf:type <http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11#Album>
    }
""")

for row in all_albums:
  print(row['album'])

http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/TODD_-_Акт_1._«Праздник_Крови»
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/TODD_-_Акт_2._«На_Краю»
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/Акустический_альбом
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/Бунт_на_корабле
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/Жаль_-_нет_ружья!
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/Как_в_старой_сказке
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/Камнем_по_Голове
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/Король_и_Шут
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/Продавец_Кошмаров
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/Театр_Демона
http://www

In [210]:
# Посмотрим на список треков с альбома "Камнем по Голове"

some_tracks_v1 =  g.query("""

    PREFIX kish: <http://www.semanticweb.org/mosto/ontologies/kish#>
    PREFIX album_1: <http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/>

    SELECT ?track
    WHERE {
      album_1:Камнем_по_Голове
      kish:contains
      ?track
    }
""")

for row in some_tracks_v1:
  print(row['track'])

http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Смельчак_и_Ветер
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Проказник_Скоморох
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Верная_Жена
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Садовник
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Блуждают_Тени
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Внезапная_Голова
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Шар_Голубой
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Злодей_и_Шапка
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/От_Женщин_Кругом_Голова
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Рыбак
http://www.semanticweb.org/mosto/ontologies/2023/10/unt

In [211]:
# Посмотрим на список треков с альбома "Король и Шут"

some_tracks_v2 =  g.query("""

    PREFIX kish: <http://www.semanticweb.org/mosto/ontologies/kish#>
    PREFIX album_1: <http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/albums/>

    SELECT ?track
    WHERE {
      album_1:Король_и_Шут
      kish:contains
      ?track
    }
""")

for row in some_tracks_v2:
  print(row['track'])

http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Король_и_Шут
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Два_друга
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Сапоги
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Охотник
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Паника_в_селе
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Истинный_убийца
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Лесник
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Помоги_мне!
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Воспоминания_о_Мёртовой_Женщине
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-ontology-11/tracks/Кукольный_Театр
http://www.semanticweb.org/mosto/ontologies/2023/10/untitled-onto